In [291]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import sklearn.metrics as m
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
import pickle

In [8]:
df = pd.read_csv('data/final.csv')
df.head(2)

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,Возраст дома,WORK_NAME_TARGET
0,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,12
1,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,15


In [9]:
df.columns

Index(['Серия проекта', 'Количество этажей', 'Количество подъездов',
       'Общая площадь', 'Общая площадь жилых помещений',
       'Общая площадь нежилых помещений', 'Материал стен',
       'Количество пассажирских лифтов', 'Количество грузопассажирских лифтов',
       'Статус МКД', 'Количество грузовых лифтов',
       'Месяц создания во внешней системе',
       'Количество обращений за последний месяц',
       'Среднемесячное количество обращений', 'Возраст дома',
       'WORK_NAME_TARGET'],
      dtype='object')

In [293]:
df.UNOM.value_counts()

24239    2610
2930     1141
64039    1012
24750     930
15109     819
         ... 
18528       4
24135       2
27141       2
21908       2
28691       2
Name: UNOM, Length: 371, dtype: int64

In [294]:
len(df.WORK_NAME.value_counts())

19

In [295]:
df.head(2)

,Год постройки,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,UNOM,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,WORK_NAME,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START
0,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем электро...,12,423.0,0.260901,3
1,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,"ремонт подвальных помещений, относящихся к общ...",12,423.0,0.260901,5


In [296]:
df['Возраст дома'] = 2022 - df['Год постройки']
df

,Год постройки,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,UNOM,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,WORK_NAME,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START,Возраст дома
0,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем электро...,12,423.0,0.260901,3,49.0
1,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,"ремонт подвальных помещений, относящихся к общ...",12,423.0,0.260901,5,49.0
2,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,"ремонт подъездов, направленный на восстановлен...",12,423.0,0.260901,5,49.0
3,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем водоотв...,12,423.0,0.260901,3,49.0
4,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,"замена оконных блоков, расположенных в помещен...",12,423.0,0.260901,1,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46425,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,ремонт крыши,12,126.0,0.067783,11,47.0
46426,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,"ремонт подвальных помещений, относящихся к общ...",12,126.0,0.067783,5,47.0
46427,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,ремонт фасадов,12,126.0,0.067783,1,47.0
46428,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,ремонт внутридомовых инженерных систем теплосн...,12,126.0,0.067783,4,47.0


In [297]:
#кодирование таргета
encoder = LabelEncoder()
df['WORK_NAME_TARGET'] = encoder.fit_transform(df.WORK_NAME)
df

,Год постройки,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,...,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,WORK_NAME,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START,Возраст дома,WORK_NAME_TARGET
0,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,...,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем электро...,12,423.0,0.260901,3,49.0,12
1,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,...,в эксплуатации,0.0,2022-12-02,"ремонт подвальных помещений, относящихся к общ...",12,423.0,0.260901,5,49.0,15
2,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,...,в эксплуатации,0.0,2022-12-02,"ремонт подъездов, направленный на восстановлен...",12,423.0,0.260901,5,49.0,16
3,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,...,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем водоотв...,12,423.0,0.260901,3,49.0,3
4,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,...,в эксплуатации,0.0,2022-12-02,"замена оконных блоков, расположенных в помещен...",12,423.0,0.260901,1,49.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46425,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,...,в эксплуатации,0.0,2022-12-20,ремонт крыши,12,126.0,0.067783,11,47.0,13
46426,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,...,в эксплуатации,0.0,2022-12-20,"ремонт подвальных помещений, относящихся к общ...",12,126.0,0.067783,5,47.0,15
46427,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,...,в эксплуатации,0.0,2022-12-20,ремонт фасадов,12,126.0,0.067783,1,47.0,18
46428,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,...,в эксплуатации,0.0,2022-12-20,ремонт внутридомовых инженерных систем теплосн...,12,126.0,0.067783,4,47.0,8


In [298]:
work_name = df[['WORK_NAME', 'WORK_NAME_TARGET']].groupby('WORK_NAME_TARGET')['WORK_NAME'].agg(lambda x: x)
work_name_key = dict()

In [299]:
work_name.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], dtype='int64', name='WORK_NAME_TARGET')

In [300]:
df = df.drop(['Год постройки', 'WORK_NAME'], axis=1)
df.head()

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,UNOM,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START,Возраст дома,WORK_NAME_TARGET
0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,12,423.0,0.260901,3,49.0,12
1,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,12,423.0,0.260901,5,49.0,15
2,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,12,423.0,0.260901,5,49.0,16
3,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,12,423.0,0.260901,3,49.0,3
4,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,12,423.0,0.260901,1,49.0,1


In [301]:
df = df.set_index('UNOM')
df

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START,Возраст дома,WORK_NAME_TARGET
UNOM,,,,,,,,,,,,,,,,,,
22830,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,в эксплуатации,0.0,2022-12-02,12,423.0,0.260901,3,49.0,12
22830,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,в эксплуатации,0.0,2022-12-02,12,423.0,0.260901,5,49.0,15
22830,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,в эксплуатации,0.0,2022-12-02,12,423.0,0.260901,5,49.0,16
22830,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,в эксплуатации,0.0,2022-12-02,12,423.0,0.260901,3,49.0,3
22830,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,в эксплуатации,0.0,2022-12-02,12,423.0,0.260901,1,49.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28393,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,в эксплуатации,0.0,2022-12-20,12,126.0,0.067783,11,47.0,13
28393,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,в эксплуатации,0.0,2022-12-20,12,126.0,0.067783,5,47.0,15
28393,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,в эксплуатации,0.0,2022-12-20,12,126.0,0.067783,1,47.0,18


In [302]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46430 entries, 22830 to 28393
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Серия проекта                            46430 non-null  object 
 1   Количество этажей                        46430 non-null  float64
 2   Количество подъездов                     46430 non-null  float64
 3   Общая площадь                            46430 non-null  float64
 4   Общая площадь жилых помещений            46430 non-null  float64
 5   Общая площадь нежилых помещений          46430 non-null  float64
 6   Материал стен                            46430 non-null  object 
 7   Количество пассажирских лифтов           46430 non-null  float64
 8   Количество грузопассажирских лифтов      46430 non-null  float64
 9   Статус МКД                               46430 non-null  object 
 10  Количество грузовых лифтов               4

In [303]:
#замена данных на инт
for col in ['Количество этажей', 'Количество подъездов', 'Количество пассажирских лифтов',
         'Количество грузопассажирских лифтов', 'Количество грузовых лифтов']:
    df.loc[df[col] == 'нет данных', col] = '-1'
    df[col] = df[col].astype('float32').astype('int32')

In [304]:
#замена данных на флоат
for col in ['Общая площадь', 'Общая площадь жилых помещений', 'Общая площадь нежилых помещений']:
    df.loc[df[col] == 'нет данных', col] = '-1'
    df[col] = df[col].astype('float32').astype('int32')

In [305]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46430 entries, 22830 to 28393
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Серия проекта                            46430 non-null  object 
 1   Количество этажей                        46430 non-null  int32  
 2   Количество подъездов                     46430 non-null  int32  
 3   Общая площадь                            46430 non-null  int32  
 4   Общая площадь жилых помещений            46430 non-null  int32  
 5   Общая площадь нежилых помещений          46430 non-null  int32  
 6   Материал стен                            46430 non-null  object 
 7   Количество пассажирских лифтов           46430 non-null  int32  
 8   Количество грузопассажирских лифтов      46430 non-null  int32  
 9   Статус МКД                               46430 non-null  object 
 10  Количество грузовых лифтов               4

In [306]:
df.loc[df['Серия проекта'] == 'нет данных', 'Серия проекта'] = 'нет серии проекта'
df.loc[df['Материал стен'] == 'нет данных', 'Материал стен'] = 'нет информации по материалам стен'

In [307]:
df

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START,Возраст дома,WORK_NAME_TARGET
UNOM,,,,,,,,,,,,,,,,,,
22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,2022-12-02,12,423.0,0.260901,3,49.0,12
22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,2022-12-02,12,423.0,0.260901,5,49.0,15
22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,2022-12-02,12,423.0,0.260901,5,49.0,16
22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,2022-12-02,12,423.0,0.260901,3,49.0,3
22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,2022-12-02,12,423.0,0.260901,1,49.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28393,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,в эксплуатации,0,2022-12-20,12,126.0,0.067783,11,47.0,13
28393,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,в эксплуатации,0,2022-12-20,12,126.0,0.067783,5,47.0,15
28393,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,в эксплуатации,0,2022-12-20,12,126.0,0.067783,1,47.0,18


In [308]:
df = df.drop(['Дата создания во внешней системе', 'PLAN_MONTH_START'], axis=1)
df

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,Возраст дома,WORK_NAME_TARGET
UNOM,,,,,,,,,,,,,,,,
22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,12
22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,15
22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,16
22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,3
22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28393,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,в эксплуатации,0,12,126.0,0.067783,47.0,13
28393,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,в эксплуатации,0,12,126.0,0.067783,47.0,15
28393,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,в эксплуатации,0,12,126.0,0.067783,47.0,18


In [309]:
#сохранение обучающего датафрейма
df.to_csv('df_train_pred_work.csv', index=False)

## Обучение модели

In [321]:
x_train, x_test, y_train, y_test = train_test_split(\
    df.drop(['WORK_NAME_TARGET'], axis = 1), df['WORK_NAME_TARGET'], test_size = 0.30, random_state = 13)
x_test

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,Возраст дома
UNOM,,,,,,,,,,,,,,,
22822,П-30,12,9,24843,24506,337,панельные,18,0,в эксплуатации,0,12,217.0,0.282217,47.0
26901,И-209А,14,1,4713,4062,651,из железобетонных сегментов,2,0,в эксплуатации,0,12,123.0,0.035627,50.0
24611,П-29,9,5,8334,7790,544,кирпичные,5,0,в эксплуатации,0,12,260.0,0.374543,47.0
367,II-68-02/16М,16,2,10791,10784,7,из железобетонных сегментов,2,2,в эксплуатации,0,12,792.0,0.426797,43.0
6314,нет серии проекта,12,2,7542,7528,14,из железобетонных сегментов,4,0,в эксплуатации,0,12,20.0,0.099391,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30166,П-49 Д,9,4,7254,7254,0,панельные,4,0,в эксплуатации,0,12,162.0,0.072351,50.0
6332,нет серии проекта,5,4,3965,2588,1377,кирпичные,0,0,в эксплуатации,0,12,174.0,0.085505,58.0
18928,И-209А,14,1,4260,4260,0,из железобетонных сегментов,2,0,в эксплуатации,0,12,294.0,0.129172,51.0


In [322]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 2326 to 64039
Data columns (total 16 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Серия проекта                            1000 non-null   object 
 1   Количество этажей                        1000 non-null   int32  
 2   Количество подъездов                     1000 non-null   int32  
 3   Общая площадь                            1000 non-null   int32  
 4   Общая площадь жилых помещений            1000 non-null   int32  
 5   Общая площадь нежилых помещений          1000 non-null   int32  
 6   Материал стен                            1000 non-null   object 
 7   Количество пассажирских лифтов           1000 non-null   int32  
 8   Количество грузопассажирских лифтов      1000 non-null   int32  
 9   Статус МКД                               1000 non-null   object 
 10  Количество грузовых лифтов               100

In [323]:
x_test

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,Возраст дома
UNOM,,,,,,,,,,,,,,,
22822,П-30,12,9,24843,24506,337,панельные,18,0,в эксплуатации,0,12,217.0,0.282217,47.0
26901,И-209А,14,1,4713,4062,651,из железобетонных сегментов,2,0,в эксплуатации,0,12,123.0,0.035627,50.0
24611,П-29,9,5,8334,7790,544,кирпичные,5,0,в эксплуатации,0,12,260.0,0.374543,47.0
367,II-68-02/16М,16,2,10791,10784,7,из железобетонных сегментов,2,2,в эксплуатации,0,12,792.0,0.426797,43.0
6314,нет серии проекта,12,2,7542,7528,14,из железобетонных сегментов,4,0,в эксплуатации,0,12,20.0,0.099391,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30166,П-49 Д,9,4,7254,7254,0,панельные,4,0,в эксплуатации,0,12,162.0,0.072351,50.0
6332,нет серии проекта,5,4,3965,2588,1377,кирпичные,0,0,в эксплуатации,0,12,174.0,0.085505,58.0
18928,И-209А,14,1,4260,4260,0,из железобетонных сегментов,2,0,в эксплуатации,0,12,294.0,0.129172,51.0


In [ ]:
'Серия проекта''Количество этажей''Количество подъездов'

In [1]:
import pandas as pd

In [3]:
pred  = pd.read_csv("data/pred.csv")
pred.head()

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,Возраст дома,WORK_NAME_TARGET
0,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,12
1,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,15
2,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,16
3,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,3
4,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,12,423.0,0.260901,49.0,1


In [4]:
pred.columns

Index(['Серия проекта', 'Количество этажей', 'Количество подъездов',
       'Общая площадь', 'Общая площадь жилых помещений',
       'Общая площадь нежилых помещений', 'Материал стен',
       'Количество пассажирских лифтов', 'Количество грузопассажирских лифтов',
       'Статус МКД', 'Количество грузовых лифтов',
       'Месяц создания во внешней системе',
       'Количество обращений за последний месяц',
       'Среднемесячное количество обращений', 'Возраст дома',
       'WORK_NAME_TARGET'],
      dtype='object')

In [ ]:
["Количество обращений за последний месяц",
                "Среднее количество обращений за месяц", "Год постройки",
                'Серия проекта', 'Количество этажей',
                'Количество подъездов', 'Общая площадь',
                'Общая площадь жилых помещений', 'Общая площадь нежилых помещений',
                'Материал стен', 'Количество пассажирских лифтов',
                'Количество грузопассажирских лифтов', 'Статус МКД',
                'Количество грузовых лифтов']

In [324]:
#сохранение тестового датафрейма
x_test.to_csv('df_test_pred_work.csv')

In [325]:
cat_features = ['Серия проекта', 'Количество этажей', 'Количество подъездов',
               'Материал стен', 'Статус МКД']

In [326]:
train_pool = Pool(x_train, y_train, cat_features=cat_features)

In [327]:
%%time
model = CatBoostClassifier(loss_function='MultiClass', random_seed=13)
model.fit(train_pool, verbose = 100)
pred = model.predict(x_test)

Learning rate set to 0.0777
0:	learn: 2.8631247	total: 84.4ms	remaining: 1m 24s
100:	learn: 1.6506578	total: 7.87s	remaining: 1m 10s
200:	learn: 1.1639864	total: 16.5s	remaining: 1m 5s
300:	learn: 0.8458967	total: 25.1s	remaining: 58.3s
400:	learn: 0.6405013	total: 33.4s	remaining: 49.9s
500:	learn: 0.4967687	total: 41.5s	remaining: 41.3s
600:	learn: 0.3979419	total: 49.9s	remaining: 33.1s
700:	learn: 0.3300370	total: 58s	remaining: 24.8s
800:	learn: 0.2790083	total: 1m 6s	remaining: 16.6s
900:	learn: 0.2384902	total: 1m 17s	remaining: 8.47s
999:	learn: 0.2072035	total: 1m 28s	remaining: 0us
CPU times: user 5min 5s, sys: 1.84 s, total: 5min 7s
Wall time: 1min 28s


In [328]:
print('F1 score -', f1_score(y_test, pred, average=None))

F1 score - [0.73267327 0.         0.         0.07142857 0.         0.
 0.10810811 0.04651163 0.06153846 0.         0.10989011 0.11764706
 0.         0.         0.         0.16393443 0.         0.
 0.12307692]


In [329]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.65      0.84      0.73        44
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00         2
           3       0.20      0.04      0.07        23
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           6       0.15      0.08      0.11        24
           7       0.03      0.07      0.05        14
           8       0.06      0.06      0.06        32
           9       0.00      0.00      0.00         1
          10       0.08      0.18      0.11        28
          11       0.09      0.17      0.12         6
          12       0.00      0.00      0.00         9
          13       0.00      0.00      0.00        26
          14       0.00      0.00      0.00         8
          15       0.18      0.15      0.16        33
          16       0.00      0.00      0.00         9
          17       0.00    

/home/localadmin/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/localadmin/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/localadmin/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Список работ с вероятностями

In [330]:
pred_proba = model.predict_proba(x_test)
pred_proba

array([[1.93500074e-03, 3.33058582e-03, 4.71253675e-03, ...,
        1.65658016e-03, 2.70380730e-03, 1.11266103e-02],
       [8.50206650e-03, 1.42998305e-02, 3.08489782e-03, ...,
        5.09682729e-03, 1.04089306e-03, 1.74604252e-01],
       [1.22914738e-02, 1.03305867e-02, 2.18828518e-03, ...,
        5.60117943e-03, 2.25491767e-03, 3.17435775e-02],
       ...,
       [8.04973884e-04, 4.67578364e-03, 1.01739106e-03, ...,
        2.30263225e-03, 7.69520996e-04, 6.96553998e-02],
       [5.64024227e-02, 6.31333202e-03, 2.13895587e-03, ...,
        5.50829101e-03, 7.91318922e-04, 5.96471888e-02],
       [8.52139945e-01, 1.53346895e-03, 7.19605050e-04, ...,
        5.93468242e-03, 4.03428037e-04, 5.38896735e-02]])

In [331]:
len(pred_proba)

300

In [332]:
#получение json словаря
with open('work_name_key.json') as f:
    templates = json.load(f)

print(templates)

{'0': 'замена лифтового оборудования', '1': 'замена оконных блоков, расположенных в помещениях общего пользования', '2': 'ремонт внутреннего водостока', '3': 'ремонт внутридомовых инженерных систем водоотведения (канализации) (выпуски и сборные трубопроводы)', '4': 'ремонт внутридомовых инженерных систем водоотведения (канализации) (стояки)', '5': 'ремонт внутридомовых инженерных систем газоснабжения', '6': 'ремонт внутридомовых инженерных систем горячего водоснабжения (разводящие магистрали)', '7': 'ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)', '8': 'ремонт внутридомовых инженерных систем теплоснабжения (разводящие магистрали)', '9': 'ремонт внутридомовых инженерных систем теплоснабжения (стояки)', '10': 'ремонт внутридомовых инженерных систем холодного водоснабжения (разводящие магистрали)', '11': 'ремонт внутридомовых инженерных систем холодного водоснабжения (стояки)', '12': 'ремонт внутридомовых инженерных систем электроснабжения', '13': 'ремонт крыши', 

In [ ]:
#Вывод предикта с вероятностью и наименванием
df_final = pd.DataFrame(data=pred_proba, columns=['Вероятность'])
df_final['Наименование работ'] = templates
df_final['Наименование работ'] = df_final['Наименование работ'].map(templates)
df_final = df_final.sort_values('Вероятность', ascending=False)
df_final

In [ ]:
model

In [24]:
#сохраняем моделb lgbm
with open('jkh_catboost.pkl', 'wb') as file:
    pickle.dump(model, file)